# Azure Service Bus (Queues and Topics/Pub-Sub)

In [ ]:
# The main difference between Azure Event Hubs and Service Bus is that Event Hubs is designed for high-flow analytics types of events, 
# such as telemetry data or clickstream analysis. Service Bus is more suitable for scenarios that require reliable messaging, 
# such as business workflows or integration with third-party systems.

# A service bus queue is a unidirectional channel that holds messages until a single consumer retrieves them. 
# - provides a one-to-one message delivery model. 
# - useful for scenarios where you need to ensure that each message is processed only once by a single consumer1.

# A service bus topic is a unidirectional channel that publishes messages to multiple subscriptions.
# - provides a one-to-many message delivery model. 
# - useful for scenarios where you need to scale to large numbers of recipients or enable message filtering and subscription.

# Queue:
# Sender --> Queue --> Receiver

# Topic:
# Sender --> Topic --> Subscription 1 --> Receiver 1
#                  --> Subscription 2 --> Receiver 2
#                  --> Subscription 3 --> Receiver 3

In [ ]:
import time
import json

## Azure Authentication

In [ ]:
# install azure cli tools: https://learn.microsoft.com/en-us/cli/azure/install-azure-cli
# install python packages: pip install azure-cli azure-identity azure-servicebus

In [ ]:
# service principal: https://learn.microsoft.com/en-us/azure/developer/python/sdk/authentication-local-development-service-principal
# import os
# os.environ["AZURE_CLIENT_ID"] = "YOUR_CLIENT_ID"
# os.environ["AZURE_TENANT_ID"] = "YOUR_TENANT_ID"
# os.environ["AZURE_CLIENT_SECRET"] = "YOUR_CLIENT_SECRET"

In [ ]:
# alternative to service principal: interactive web login via cli
!az login

In [ ]:
# set subscription
# !az account show/list/set
from azure.cli.core import get_default_cli
get_default_cli().invoke(["account", "show"])

In [ ]:
# azure credential object 
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

## Azure Service Bus: Queue

In [ ]:
# get connection string via azure portal -> service bus -> shared access policies -> add
CONNECTION_STRING = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
SERVICE_BUS_QUEUE_NAME = "XXXXXXXXXXXXXXXXXXX"

In [ ]:
from azure.servicebus import ServiceBusClient, ServiceBusMessage

# send to queue
with ServiceBusClient.from_connection_string(CONNECTION_STRING) as client:
    with client.get_queue_sender(SERVICE_BUS_QUEUE_NAME) as sender:
        # send messages
        messages = [
            ServiceBusMessage(json.dumps({"message": "test 1"})),
            ServiceBusMessage(json.dumps({"message": "test 2"})),
            ServiceBusMessage(json.dumps({"message": "test 3"}))
        ]
        try:
            sender.send_messages(messages)
            print(f"sent sucessfully {time.ctime()}")
        except Exception as e:
            print(f"failed {time.ctime()}, exception: {e}")

In [ ]:
# receive from queue
with ServiceBusClient.from_connection_string(CONNECTION_STRING) as client:
    with client.get_queue_receiver(SERVICE_BUS_QUEUE_NAME) as receiver:
        messages = receiver.receive_messages(max_message_count=10, max_wait_time=1)
        for message in messages:
            message_json = json.loads(str(message))
            print(f"received: {message_json}")
            # remove from queue
            receiver.complete_message(message)

## Azure Service Bus: Topics (Pub-Sub)

In [ ]:
# get connection string via azure portal -> service bus -> shared access policies -> add
CONNECTION_STRING = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
SERVICE_BUS_TOPIC_NAME = "XXXXXXXXXXXXXXX"
SERVICE_BUS_SUB_NAME = "XXXXXXXXXXXXX"

In [ ]:
from azure.servicebus import ServiceBusClient, ServiceBusMessage

# send to topic
with ServiceBusClient.from_connection_string(CONNECTION_STRING) as client:
    with client.get_topic_sender(topic_name=SERVICE_BUS_TOPIC_NAME) as sender:
        # send messages
        messages = [
            ServiceBusMessage(json.dumps({"message": "test 1"})),
            ServiceBusMessage(json.dumps({"message": "test 2"})),
            ServiceBusMessage(json.dumps({"message": "test 3"}))
        ]
        try:
            sender.send_messages(messages)
            print(f"sent sucessfully {time.ctime()}")
        except Exception as e:
            print(f"failed {time.ctime()}, exception: {e}")

In [ ]:
# receive from topic
with ServiceBusClient.from_connection_string(CONNECTION_STRING) as client:
    with client.get_subscription_receiver(topic_name=SERVICE_BUS_TOPIC_NAME, subscription_name=SERVICE_BUS_SUB_NAME) as receiver:
        messages = receiver.receive_messages(max_message_count=10, max_wait_time=1)
        for message in messages:
            message_json = json.loads(str(message))
            print(f"received: {message_json}")
            # mark as processed
            receiver.complete_message(message)